In [1]:
# import os
# import numpy as np
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.applications import EfficientNetB3
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
# from tensorflow.keras.regularizers import l2
# from tensorflow.keras.losses import CategoricalCrossentropy
# from sklearn.metrics import classification_report, confusion_matrix
# from sklearn.utils.class_weight import compute_class_weight
# import tensorflow as tf

# # Enable mixed precision training
# policy = tf.keras.mixed_precision.Policy('mixed_float16')
# tf.keras.mixed_precision.set_global_policy(policy)

In [2]:
# # Paths
# base_path = "/kaggle/input/dataset-for-crop-pest-and-disease-detection/Dataset for Crop Pest and Disease Detection"
# train_dir = os.path.join(base_path, "CCMT Dataset-Augmented/Cashew/train_set")
# test_dir = os.path.join(base_path, "CCMT Dataset-Augmented/Cashew/test_set")

# # Parameters
# img_size = (224, 224)
# batch_size = 64
# epochs = 35

# # Augmentation
# train_datagen = ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=35,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     vertical_flip=True,
#     brightness_range=[0.8, 1.2],
#     fill_mode='nearest'
# )

# test_datagen = ImageDataGenerator(rescale=1./255)

# # Data Loaders
# train_data = train_datagen.flow_from_directory(
#     train_dir,
#     target_size=img_size,
#     batch_size=batch_size,
#     class_mode='categorical'
# )

# test_data = test_datagen.flow_from_directory(
#     test_dir,
#     target_size=img_size,
#     batch_size=batch_size,
#     class_mode='categorical',
#     shuffle=False
# )

# # Model
# base_model = EfficientNetB3(
#     input_shape=img_size + (3,),
#     include_top=False,
#     weights='imagenet'
# )

# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = BatchNormalization()(x)
# x = Dense(768, activation='relu', kernel_regularizer=l2(1e-4))(x)
# x = Dropout(0.4)(x)  # Increased dropout
# x = BatchNormalization()(x)
# output = Dense(train_data.num_classes, activation='softmax', dtype='float32')(x)
# model = Model(inputs=base_model.input, outputs=output)

# # Freeze base model initially
# base_model.trainable = False

# # Learning rate schedule
# def lr_schedule(epoch):
#     if epoch < 15:
#         return 3e-4
#     elif epoch < 25:
#         return 1e-4
#     else:
#         return 5e-5

# model.compile(
#     optimizer=Adam(3e-4),
#     loss=CategoricalCrossentropy(label_smoothing=0.1),
#     metrics=['accuracy']
# )

# # Callbacks
# checkpoint = ModelCheckpoint(
#     "cashew_best.weights.h5",
#     monitor='val_accuracy',
#     save_best_only=True,
#     save_weights_only=True
# )
# early_stop = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
# lr_scheduler = LearningRateScheduler(lr_schedule)

# # Phase 1: Frozen base model
# history1 = model.fit(
#     train_data,
#     epochs=15,
#     validation_data=test_data,
#     callbacks=[checkpoint, early_stop, reduce_lr, lr_scheduler],
#     verbose=1
# )

# # Phase 2: Unfreeze some layers
# base_model.trainable = True
# for layer in base_model.layers[:100]:
#     layer.trainable = False
    
# model.compile(
#     optimizer=Adam(1e-4),
#     loss=CategoricalCrossentropy(label_smoothing=0.1),
#     metrics=['accuracy']
# )

# history2 = model.fit(
#     train_data,
#     initial_epoch=15,
#     epochs=epochs,
#     validation_data=test_data,
#     callbacks=[checkpoint, early_stop, reduce_lr, lr_scheduler],
#     verbose=1
# )

# # Evaluation
# model.load_weights("cashew_best.weights.h5")
# loss, acc = model.evaluate(test_data, verbose=0)
# print(f"\n✅ Cashew Test Accuracy: {acc*100:.2f}%")

# # Classification Report
# Y_pred = model.predict(test_data)
# y_pred = np.argmax(Y_pred, axis=1)
# print("\nClassification Report:")
# print(classification_report(test_data.classes, y_pred, target_names=list(test_data.class_indices.keys())))

In [3]:
# # Paths
# base_path = "/kaggle/input/dataset-for-crop-pest-and-disease-detection/Dataset for Crop Pest and Disease Detection"
# train_dir = os.path.join(base_path, "CCMT Dataset-Augmented/Cassava/train_set")
# test_dir = os.path.join(base_path, "CCMT Dataset-Augmented/Cassava/test_set")

# # Handle duplicate classes
# valid_classes = [c for c in os.listdir(train_dir) if c in os.listdir(test_dir)]

# # Parameters
# img_size = (224, 224)
# batch_size = 64
# epochs = 40

# # Advanced Augmentation
# train_datagen = ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=40,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     vertical_flip=True,
#     brightness_range=[0.8, 1.2],
#     channel_shift_range=30,
#     fill_mode='nearest'
# )

# test_datagen = ImageDataGenerator(rescale=1./255)

# # Data Loaders
# train_data = train_datagen.flow_from_directory(
#     train_dir,
#     target_size=img_size,
#     batch_size=batch_size,
#     class_mode='categorical',
#     classes=valid_classes
# )

# test_data = test_datagen.flow_from_directory(
#     test_dir,
#     target_size=img_size,
#     batch_size=batch_size,
#     class_mode='categorical',
#     shuffle=False,
#     classes=valid_classes
# )

# # Model with Stochastic Depth
# base_model = EfficientNetB3(
#     input_shape=img_size + (3,),
#     include_top=False,
#     weights='imagenet'
# )

# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = BatchNormalization()(x)
# x = Dense(1024, activation='relu', kernel_regularizer=l2(1e-4))(x)
# x = Dropout(0.5)(x)
# x = BatchNormalization()(x)
# output = Dense(train_data.num_classes, activation='softmax', dtype='float32')(x)
# model = Model(inputs=base_model.input, outputs=output)
# # Class Weights
# class_weights = compute_class_weight('balanced', classes=np.unique(train_data.classes), y=train_data.classes)
# class_weights = dict(enumerate(class_weights))

# # Focal Loss
# def focal_loss(gamma=2.0, alpha=0.25):
#     def loss_fn(y_true, y_pred):
#         epsilon = tf.keras.backend.epsilon()
#         y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
#         cross_entropy = -y_true * tf.math.log(y_pred)
#         loss = alpha * tf.pow(1. - y_pred, gamma) * cross_entropy
#         return tf.reduce_mean(loss)
#     return loss_fn

# model.compile(
#     optimizer=Adam(2e-4),
#     loss=focal_loss(),
#     metrics=['accuracy']
# )

# # Callbacks
# checkpoint = ModelCheckpoint(
#     "cassava_best.weights.h5",
#     monitor='val_accuracy',
#     save_best_only=True,
#     save_weights_only=True
# )
# early_stop = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-6)

# # Training
# history = model.fit(
#     train_data,
#     epochs=epochs,
#     validation_data=test_data,
#     callbacks=[checkpoint, early_stop, reduce_lr],
#     class_weight=class_weights,
#     verbose=1
# )

# # Evaluation
# model.load_weights("cassava_best.weights.h5")
# loss, acc = model.evaluate(test_data, verbose=0)
# print(f"\n✅ Cassava Test Accuracy: {acc*100:.2f}%")

# # Classification Report
# Y_pred = model.predict(test_data)
# y_pred = np.argmax(Y_pred, axis=1)
# print("\nClassification Report:")
# print(classification_report(test_data.classes, y_pred, target_names=list(test_data.class_indices.keys())))

In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.regularizers import l2
from tensorflow.keras.losses import CategoricalCrossentropy
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf

# Enable mixed precision training
policy = tf.keras.mixed_precision.Policy('mixed_float16')
tf.keras.mixed_precision.set_global_policy(policy)

# Paths
base_path = "/kaggle/input/dataset-for-crop-pest-and-disease-detection/Dataset for Crop Pest and Disease Detection"
train_dir = os.path.join(base_path, "CCMT Dataset-Augmented/Tomato/train_set")
test_dir = os.path.join(base_path, "CCMT Dataset-Augmented/Tomato/test_set")

# Optimized Parameters
img_size = (224, 224)  # Slightly larger than 192
batch_size = 48  # Reduced for better memory usage
epochs = 50  # More epochs for better convergence

# Enhanced Augmentation - Tomato-specific
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='reflect'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Data Loading
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Enhanced Class Weighting
class_weights = compute_class_weight('balanced', classes=np.unique(train_data.classes), y=train_data.classes)
class_weights = {i: weight*1.5 for i, weight in enumerate(class_weights)}

# Model Architecture
base_model = MobileNetV2(
    input_shape=img_size + (3,),
    include_top=False,
    weights='imagenet',
    alpha=1.0  # Using full width version
)

# Progressive Unfreezing Strategy
base_model.trainable = True
for layer in base_model.layers[:120]:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(768, activation='relu', kernel_regularizer=l2(1e-4))(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(384, activation='relu', kernel_regularizer=l2(1e-4))(x)
x = Dropout(0.3)(x)
output = Dense(train_data.num_classes, activation='softmax', dtype='float32')(x)
model = Model(inputs=base_model.input, outputs=output)

# Custom Learning Rate Schedule
def lr_schedule(epoch):
    if epoch < 15:
        return 3e-4
    elif epoch < 30:
        return 1e-4
    elif epoch < 40:
        return 5e-5
    else:
        return 1e-5

model.compile(
    optimizer=Adam(3e-4),
    loss=CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

# Enhanced Callbacks
checkpoint = ModelCheckpoint(
    "tomato_mobilenet_best.weights.h5",
    monitor='val_accuracy',
    save_best_only=True,
    save_weights_only=True,
    mode='max'
)

early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=15,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-6
)

lr_scheduler = LearningRateScheduler(lr_schedule)

# Training
history = model.fit(
    train_data,
    epochs=epochs,
    validation_data=test_data,
    callbacks=[checkpoint, early_stop, reduce_lr, lr_scheduler],
    class_weight=class_weights,
    verbose=1
)

# Evaluation
model.load_weights("tomato_mobilenet_best.weights.h5")
loss, acc = model.evaluate(test_data, verbose=0)
print(f"\n✅ Tomato Test Accuracy: {acc*100:.2f}%")

# Classification Report
Y_pred = model.predict(test_data)
y_pred = np.argmax(Y_pred, axis=1)
print("\nClassification Report:")
print(classification_report(test_data.classes, y_pred, target_names=list(test_data.class_indices.keys())))

# Confusion Matrix
plt.figure(figsize=(10,8))
conf_mat = confusion_matrix(test_data.classes, y_pred)
sns.heatmap(conf_mat, annot=True, fmt='d', 
            xticklabels=test_data.class_indices.keys(),
            yticklabels=test_data.class_indices.keys())
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

2025-07-19 11:19:53.617149: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752923993.825129      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752923993.885166      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found 21723 images belonging to 5 classes.
Found 5445 images belonging to 5 classes.


I0000 00:00:1752924028.317958      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


...

  1/453 ━━━━━━━━━━━━━━━━━━━━ 3:52:06 31s/step - accuracy: 0.1875 - loss: 3.6048

I0000 00:00:1752924063.543250      72 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


453/453 ━━━━━━━━━━━━━━━━━━━━ 545s 1s/step - accuracy: 0.3829 - loss: 2.4769 - val_accuracy: 0.3390 - val_loss: 4.7535 - learning_rate: 3.0000e-04
Epoch 2/50
453/453 ━━━━━━━━━━━━━━━━━━━━ 309s 681ms/step - accuracy: 0.5194 - loss: 1.9275 - val_accuracy: 0.2444 - val_loss: 3.2491 - learning_rate: 3.0000e-04
Epoch 3/50
453/453 ━━━━━━━━━━━━━━━━━━━━ 305s 674ms/step - accuracy: 0.5824 - loss: 1.7184 - val_accuracy: 0.5420 - val_loss: 1.6335 - learning_rate: 3.0000e-04
Epoch 4/50
453/453 ━━━━━━━━━━━━━━━━━━━━ 300s 661ms/step - accuracy: 0.6269 - loss: 1.6069 - val_accuracy: 0.5574 - val_loss: 1.6431 - learning_rate: 3.0000e-04
Epoch 5/50
453/453 ━━━━━━━━━━━━━━━━━━━━ 303s 668ms/step - accuracy: 0.6606 - loss: 1.5372 - val_accuracy: 0.6174 - val_loss: 1.2369 - learning_rate: 3.0000e-04
Epoch 6/50
453/453 ━━━━━━━━━━━━━━━━━━━━ 304s 671ms/step - accuracy: 0.7013 - loss: 1.4482 - val_accuracy: 0.6206 - val_loss: 1.3338 - learning_rate: 3.0000e-04
Epoch 7/50
453/453 ━━━━━━━━━━━━━━━━━━━━ 304s 670ms/ste

NameError: name 'sns' is not defined

<Figure size 1000x800 with 0 Axes>